In [51]:
import torch
import torch.nn.functional
import torch_geometric.nn 
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
import pandas as pd
import networkx as nx
import numpy as np
import tqdm
import random
from torch_geometric.nn import GATv2Conv
from torch_geometric.nn import global_mean_pool

In [3]:
pkl_graphs = pd.read_pickle('/Users/MathildeStouby/Desktop/P5 GitHub/5-semester/Momentum graphs.pkl')

In [4]:
unique_nodes = []
for graph in pkl_graphs.values():
    temp = [node for node in graph.nodes() if node not in unique_nodes]
    unique_nodes.extend(temp)

In [5]:
idx_to_pos = dict(enumerate(unique_nodes))
pos_to_idx = {pos : idx for idx, pos in idx_to_pos.items()}

In [32]:
pyg_data = []

#add node attributes
for graph in pkl_graphs.values():
    for node in unique_nodes:
        vect = np.zeros((len(unique_nodes)))
        if node not in graph.nodes:
            graph.add_node(node)
        vect[pos_to_idx[node]] = 1
        graph.nodes[node]['x'] = torch.from_numpy(vect).float()     
    data = from_networkx(graph)
    try:
        data.momentum
        pyg_data.append(data)
    except:
        print(data)
   

Data(x=[23, 23], edge_index=[2, 92], weight=[92])


In [33]:
train_idx = random.sample(range(len(pyg_data)), int(len(pyg_data) * 0.8))
test_idx = [i for i in range(len(pyg_data)) if i not in train_idx]

In [90]:
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads, v2 = True):
        super(GAT, self).__init__()
        self.layer1= GATv2Conv(input_dim, hidden_dim, heads=num_heads)
        self.layer2= GATv2Conv(hidden_dim * num_heads, hidden_dim, heads=num_heads)
        self.layer3= GATv2Conv(hidden_dim * num_heads, hidden_dim, heads=num_heads)
        self.layer4 = GATv2Conv(hidden_dim * num_heads, output_dim, heads=1, concat=False)
     

    def forward(self, input, edge_index):
        output = self.layer1(input, edge_index)
        output = self.layer2(output, edge_index)
        output = self.layer3(output, edge_index)
        output = self.layer4(output, edge_index)
        output = output.mean(dim=0) 
        return output

In [91]:
input_dim = len(unique_nodes)
lr = 0.01

gat = GAT(input_dim = input_dim, hidden_dim = 11, output_dim = 1, num_heads = 6)
optimizer = torch.optim.SGD(gat.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss()
epochs_num = 20

In [92]:
for epoch in tqdm.tqdm(range(epochs_num)):
    
    epoch_loss = 0
    for idx in tqdm.tqdm(range(len(train_idx))):
        
        input = pyg_data[idx].x
        edge_idx = pyg_data[idx].edge_index
        label = pyg_data[idx]['momentum']
    
        # Forward pass
        optimizer.zero_grad()
        
        output = gat(input, egde_idx)
        
        # Calculate loss
        loss = loss_fn(output, label)
        loss.backward()
        # optimizer.step()
        for p in gat.parameters():
            p.data.add_(p.grad.data, alpha=-lr)
        
        epoch_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(train_idx):.4f}") 
gat.eval() 

  0%|          | 0/20 [00:00<?, ?it/s]/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  5%|▌         | 1/20 [00:02<00:50,  2.63s/it]

Epoch 1, Loss: 0.0065


 10%|█         | 2/20 [00:05<00:45,  2.53s/it]

Epoch 2, Loss: 0.0065


 15%|█▌        | 3/20 [00:07<00:41,  2.45s/it]

...

 95%|█████████▌| 19/20 [00:46<00:02,  2.42s/it]

Epoch 19, Loss: 0.0062


100%|██████████| 20/20 [00:48<00:00,  2.44s/it]

Epoch 20, Loss: 0.0062


GAT(
  (layer1): GATv2Conv(23, 11, heads=6)
  (layer2): GATv2Conv(66, 11, heads=6)
  (layer3): GATv2Conv(66, 11, heads=6)
  (layer4): GATv2Conv(66, 1, heads=1)
)

In [93]:
y_pred  = []
y_true = []

with torch.no_grad():
    for idx in test_idx:
        output = gat(pyg_data[idx].x, pyg_data[idx].edge_index)
        y_pred.append(output)
        y_true.append(pyg_data[idx].momentum)
 

In [94]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_true, y_pred)

0.007573729